# Tarea 02: idf

### En esta tarea, se trabajará con un corpus de textos para obtener las palabras mas y menos relevantes a partir de un análisis por tf-idf. Deberás seguir los pasos de este notebook

In [125]:
"""
Paso 1) 
Del archivo adjunto (Questions.csv), extraer las primeras 2,000 preguntas presentes en la columna "Title", 
y generar un nuevo archivo que SOLO contenga dichas preguntas.
NOTA: El archivo es muy grande, por lo que deberás de procesarlo en tu equipo local (No usar colab), y 
generar con ayuda de un script el nuevo archivo, si llegas a tener problemas con el tipo de codificación
y se generan caracteres raros, deberás resolverlo con la codificación adecuada de lectura

Paso 2) 
Traducirlas las preguntas del nuevo archivo al idioma Español (Investiga cómo se realiza el proceso de 
traducción automático utilizando Google Sheets y Google Translate)
Se deberá de agregar una nueva columna al archivo .csv llamada "Textos_traducidos" donde se incluirán las 
traducciones de los textos originales (Incluir este nuevo .csv en la entrega de tu tarea)

Paso 3) 
Con este nuevo .csv de 2,000 textos traducidos al español, crear un dataset (de 2,000 textos) y aplicar 
OBLIGATORIAMENTE los siguientes preprocesamientos:
- Lematización de todas las palabras
- Filtrado de StopWords
- Pasar todo a minúsculas

Paso 4) 
Después de haber limpiado el dataset anterior, generar el vector de idf correspondiente a TODOS los textos
y mostrarlo en pantalla

Paso 5) Regresar al Dataset original, y remover todas aquellas palabras que contengan un valor de idf menor
al promedio de TODOS los idfs de la tabla obtenida

Paso 6)
Generar nuevamente la tabla de idf a partir de valores de idf para los textos filtrados y mostrarla en pantalla

Paso 7) Imprimir en pantalla el top de las 10 palabras MAS relevantes, y el top de las 20 palabras MENOS 
relevantes

NOTA: Recuerda que deberás de entregar 2 archivos, un .csv con los 2,000 textos originales y sus traducciones
y un .ipynb con todo el procedimiento realizado con sus respectivos comentarios, y DocStrings
IMPORTANTE: Todo el proceso deberá realizarse por medio de métodos, NO se aceptará programación estructurada,
por lo que, por ejemplo, deberá haber un método para filtrar StopWords, otro para obtener el promedio de 
idf de todo el conjunto de palabras, etc. 
"""

'\nPaso 1) \nDel archivo adjunto (Questions.csv), extraer las primeras 2,000 preguntas presentes en la columna "Title", \ny generar un nuevo archivo que SOLO contenga dichas preguntas.\nNOTA: El archivo es muy grande, por lo que deberás de procesarlo en tu equipo local (No usar colab), y \ngenerar con ayuda de un script el nuevo archivo, si llegas a tener problemas con el tipo de codificación\ny se generan caracteres raros, deberás resolverlo con la codificación adecuada de lectura\n\nPaso 2) \nTraducirlas las preguntas del nuevo archivo al idioma Español (Investiga cómo se realiza el proceso de \ntraducción automático utilizando Google Sheets y Google Translate)\nSe deberá de agregar una nueva columna al archivo .csv llamada "Textos_traducidos" donde se incluirán las \ntraducciones de los textos originales (Incluir este nuevo .csv en la entrega de tu tarea)\n\nPaso 3) \nCon este nuevo .csv de 2,000 textos traducidos al español, crear un dataset (de 2,000 textos) y aplicar \nOBLIGATORI

In [126]:
# Recuerda que todos los métodos que utilices deberán de contar con el formato DocString
# como en el ejemplo que se muestra a continuación:
# Ejemplo de formato Docstrings:

# def NombreFuncion(arg1, arg2, arg3):
#     """
#     Este método sirve para... utilizando... y devuelve...
    
#     Args:
#         string arg1: Esta es una cadena de texto que...
#         int arg 2: Es un número entero que se usa para...
#         dict arg 3: Diccionario que sirve para...

#     Returns:
#         string: Cadena del texto ya corregido...
#         int: El la cantidad de correcciones realizadas...
#     """

#     # Aquí debe de ir la lógica de la función (Después de la documentación)
#     Texto = ""
#     corr = 5
    
#   return Texto, corr

In [127]:
import pandas as pd
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import re

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/eubgo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/eubgo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/eubgo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [128]:
class TextAnalysis:
    """
    Class to analyze text data.

    Parameters:
        filepath (str): Path to the input CSV file.
    """
    def __init__(self, filepath, flag):
        """
        Initializes the TextAnalysis class with given file path, and preprocesses the data.

        Parameters:
            filepath (str): Path to the input CSV file.
        
        Returns:
            Nothing. Initializes attributes and preprocesses the data.
        """
        self.filepath = filepath
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = nltk.corpus.stopwords.words("spanish")
        self.vectorizer = TfidfVectorizer(use_idf=True)
        self.read_data()
        self.preprocess_translated_text()
        self.calculate_idf()
        if flag == False:
            self.remove_below_average_words_from_sentences()

    def read_data(self):
        """
        Reads the CSV data from the provided file path and saves the 'Translated_Texts' column to the class.

        Returns:
            Nothing. Saves the 'Translated_Texts' to the class.
        """
        self.df = pd.read_csv(self.filepath)["Translated_Texts"]
        self.df = self.df.dropna()
        
    def preprocess_translated_text(self):
        """
        Tokenizes, lemmatizes, and removes stopwords from the 'Translated_Texts' column.

        Returns:
            Nothing. Updates the 'Translated_Texts' column in the dataframe with the preprocessed text.
        """
        def process(text):
            words = nltk.word_tokenize(text)
            words = [
                self.lemmatizer.lemmatize(word.lower())
                for word in words
                if word.lower() not in self.stop_words and not (any(c.isalpha() for c in word) and any(c.isdigit() for c in word))
            ]
            return " ".join(words)
        
        def clean_char_tokens(text):
            words = nltk.word_tokenize(text)
            words = [word for word in words if len(word) > 1]
            cleaned_tokens = [re.sub(r"^\W+|\W*$", "", word) for word in words if len(word) > 1]
            return " ".join(cleaned_tokens)
        
        self.df = self.df.apply(process)
        self.df = self.df.apply(clean_char_tokens)
        self.df = self.df.dropna()

    def calculate_idf(self):
        """
        Calculates the IDF values of the words.

        Parameters: 
            Dataframe to calculate IDF

        Returns:
            Nothing. Saves the IDF values, the average IDF value, and a dataframe of words with their IDF values to the class.
        """
        self.vectorizer = TfidfVectorizer(use_idf=True)
        # Asegúrate de que cada entrada se trate como un documento separado
        tfidf_matrix = self.vectorizer.fit_transform(self.df)
        # Extrae los valores IDF y los nombres de las palabras
        self.idf_values = self.vectorizer.idf_
        self.idf_values = pd.DataFrame({"Word": self.vectorizer.get_feature_names_out(), "IDF": self.idf_values})
        
        # Calcula el valor promedio de IDF
        self.average_idf = self.idf_values["IDF"].mean()
        
        # Ordena las palabras según sus valores IDF en orden descendente
        self.df_sorted = self.idf_values.sort_values(by='IDF', ascending=False)
        
        

        
    def filter_below_average_idf(self):
        """
        Deletes the words with below average IDF values from the 'Translated_Texts' column.

        Returns:
            Nothing. Updates the 'Translated_Texts' column in the dataframe with the filtered text.
        """
        below_average = self.idf_values[self.idf_values["IDF"] < self.average_idf]
        below_average_words = below_average["Word"].str.lower().tolist()

        return below_average_words
    
    def remove_below_average_words_from_sentences(self):
        """
        Removes words with below average IDF values from the 'Translated_Texts' column.

        Returns:
            Nothing. Updates the 'Translated_Texts' column in the dataframe by removing the words.
        """
        below_average_words = self.filter_below_average_idf()

        def filter_words(text):
            return " ".join([word for word in text.split() if word.lower() not in below_average_words])

        # Filtrar las palabras por debajo del promedio en cada oración
        self.df = self.df.apply(filter_words)
        print(self.df.head())
        self.calculate_idf()


In [129]:
def extract_and_save_titles(input_file, output_file, num_rows=2000):
    """
    Extract a specified number of rows from the "Title" column of a CSV file and save to a new CSV file.

    Parameters:
        input_file (str): Path to the input CSV file.
        output_file (str): Path to save the extracted titles as a new CSV file.
        num_rows (int, optional): Number of rows to extract from the "Title" column. Default is 2000.
    """
    df = pd.read_csv(input_file, encoding="latin-1")
    df_titles = pd.DataFrame(df.head(num_rows)["Title"])
    df_titles.to_csv(output_file, index=False, encoding="UTF-8")

In [130]:
if __name__ == "__main__":
    """
    Paso 1)
    Del archivo adjunto (Questions.csv), extraer las primeras 2,000 preguntas presentes en la columna "Title",
    y generar un nuevo archivo que SOLO contenga dichas preguntas.
    NOTA: El archivo es muy grande, por lo que deberás de procesarlo en tu equipo local (No usar colab), y
    generar con ayuda de un script el nuevo archivo, si llegas a tener problemas con el tipo de codificación
    y se generan caracteres raros, deberás resolverlo con la codificación adecuada de lectura
    """
    extract_and_save_titles("Questions.csv", "Questions_2000.csv")

    """
    Paso 2) 
    Traducirlas las preguntas del nuevo archivo al idioma Español (Investiga cómo se realiza el proceso de 
    traducción automático utilizando Google Sheets y Google Translate)
    Se deberá de agregar una nueva columna al archivo .csv llamada "Textos_traducidos" donde se incluirán las 
    traducciones de los textos originales (Incluir este nuevo .csv en la entrega de tu tarea)
    """
    # In TranslatedQuestions.csv file

    """
    Paso 3) 
    Con este nuevo .csv de 2,000 textos traducidos al español, crear un dataset (de 2,000 textos) y aplicar 
    OBLIGATORIAMENTE los siguientes preprocesamientos:
    - Lematización de todas las palabras
    - Filtrado de StopWords
    - Pasar todo a minúsculas
    """
    analysis = TextAnalysis("TranslatedQuestions.csv", 1)

    """
    Paso 4) 
    Después de haber limpiado el dataset anterior, generar el vector de idf correspondiente a TODOS los textos
    y mostrarlo en pantalla
    """
    # print("Initial IDF Table:")
    # print(analysis.df_sorted) # CANDADO
    print("----------------------------------------")
    """
    Paso 5) Regresar al Dataset original, y remover todas aquellas palabras que contengan un valor de idf menor
    al promedio de TODOS los idfs de la tabla obtenida
    """
    new_anaylsis = TextAnalysis("TranslatedQuestions.csv", 0)
    
    """
    Paso 6)
    Generar nuevamente la tabla de idf a partir de valores de idf para los textos filtrados y mostrarla en pantalla
    """
    # print("Filtered IDF Table:)
    # print(new_anaylsis.df_sorted)

----------------------------------------
0               
1    previa jpeg
2               
3               
4               
Name: Translated_Texts, dtype: object
           Word       IDF
0            04  7.908255
1424      pesar  7.908255
1390  partición  7.908255
1389     parsec  7.908255
1388      parse  7.908255
...         ...       ...
1559         re  6.809643
1859        sys  6.809643
616      django  6.521961
1366         os  6.298817
1532     python  5.768189

[2085 rows x 2 columns]
